# An Intro to Polylines - The Interpolated Spline

I do a lot of work in the CAD world and one of my tools is the spline, it
essentially gives you a way to tell the computer, *"here's a bunch of points,
please draw a smooth line that passes through them"*.

The term, *"spline"*, originally came from ship builders who would use flexible
strips of wood held using pegs to draw smooth curves, but nowadays the term can
refer to a variety of curves.

In an effort to explore the mathematics behind the various types of spline, I
thought I'd try to implement them myself and add them to [`arcs`][arcs], a
Rust CAD engine I've been playing around with in my spare time.

Here are some of the splines we can choose from:

- [Hermite Spline](https://en.wikipedia.org/wiki/Cubic_Hermite_spline) - the
  mathematical function you get when trying to minimise the "tension" in a
  flexible curve passing
- [B-Spline](https://en.wikipedia.org/wiki/B-spline)
- [NURBS](https://en.wikipedia.org/wiki/Non-uniform_rational_B-spline) - The
  general form of a *B-Spline*, used all over the place in CAM
- [Bézier Curve](https://en.wikipedia.org/wiki/B%C3%A9zier_curve) - sibling of
  the *B-Spline*, often used in fonts
- [Interpolated Cubic/Quadratic Spline][wiki] - what you get after fitting a
  polynomial to the curve

As the easiest for me to wrap my head around, I thought I'd start with the
[*Interpolated Spline*][wiki]. 

I'll try not to go too deeply into the math. However, considering computational 
geometry (the study of drawing things and doing geometry inside a computer) 
is literally a field of mathematics, it's not possible to avoid it altogether.

[arcs]: https://github.com/Michael-F-Bryan/arcs
[wiki]: https://en.wikipedia.org/wiki/Polynomial_interpolation

In [14]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr"] }
:dep derive_more = "0.99.7"

use plotters::prelude::*;

In [95]:
use std::fmt::{self, Display, Formatter};
use derive_more::*;

#[derive(Copy, Clone, Debug, PartialEq, Add, Mul, Sum, AddAssign, MulAssign)]
pub struct Point {
    x: f64,
    y: f64,
}

impl Point {
    pub const fn new(x: f64, y: f64) -> Self {
        Point { x, y }
    }
    
    pub const fn zero() -> Self {
        Point::new(0.0, 0.0)
    }
    
    pub const fn tuple(self) -> (f64, f64) {
        (self.x, self.y)
    }
}

impl Display for Point {
    fn fmt(&self, f: &mut Formatter<'_>) -> fmt::Result {
        write!(f, "(")?;
        Display::fmt(&self.x, f)?;
        write!(f, ", ")?;
        Display::fmt(&self.y, f)?;
        write!(f, ")")?;

        Ok(())
    }
}

In [167]:
use plotters::coord::RangedCoordf64;

fn draw_points< 'a, P, D, F>(
    ctx: &'a mut ChartContext<'a, D, RangedCoord<RangedCoordf64, RangedCoordf64>>, 
    points: P, 
    mut getter: F) -> Result<(), Box<dyn std::error::Error + 'a>> 
where D: DrawingBackend + 'a,
      P: IntoIterator<Item = Point>,
      F: FnMut(usize, Point) -> (f64, f64),
      D::ErrorType: 'a,
{
    let mut points: Vec<_> = points.into_iter().enumerate().map(|(i, p)| getter(i, p)).collect();
    ctx.draw_series(points.iter().map(|p| Circle::new(*p, 3, RED.filled())))?;

    let path = PathElement::new(points.iter().copied().collect::<Vec<_>>(), RED.filled());
    ctx.plotting_area().draw(&path)?;
    
    Ok(())
}

## The General Idea


Say you have a bunch of points that you'd like to draw a curve for.

In [156]:
let points = vec![
    Point::new(0.0, 0.0),
    Point::new(0.7, 1.2),
    Point::new(1.1, 3.1),
    Point::new(4.0, 2.1),
    Point::new(2.5, -1.0),
];

By far the easiest way to draw this is to just connect the dots, like so.

In [157]:
evcxr_figure((640, 480), |root| {
   let mut ctx = ChartBuilder::on(&root)
    .x_label_area_size(40)
    .y_label_area_size(40)
    .build_ranged(-1.0f64.. 5.0f64, -2.0f64..4.0f64)?;
    
    ctx.configure_mesh().draw()?;
    
    fn type_name<T>(_: &T) -> &'static str {
        std::any::type_name::<T>()
    }
    println!("{}", type_name(&ctx));
    
    ctx.draw_series(points.iter().map(|p| Circle::new(p.tuple(), 3, RED.filled())))?;

    let path = PathElement::new(points.iter().map(|p| p.tuple()).collect::<Vec<_>>(), RED.filled());
    ctx.plotting_area().draw(&path)?;
        
    Ok(())
})

plotters::chart::context::ChartContext<plotters::drawing::backend_impl::svg::SVGBackend, plotters::coord::ranged::RangedCoord<plotters::coord::numeric::RangedCoordf64, plotters::coord::numeric::RangedCoordf64>>


-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0

While it may feel a bit naive, for a lot of applications this can be quite adequate.
Sure, the data might actually be a lot curvier and we're not really accounting for
trends, but if the points are close together and don't change very much, the difference 
between the "actual" curve and our linear approximation will be pretty small.

> **TIP:** TL;DR: You can brute-force the whole "accuracy" problem by just having more points
> in the input.

In [181]:
evcxr_figure((640, 480), |root| {
    let areas = root.split_evenly((2,1));
    let mut charts = Vec::new();

    for (area, name) in areas.iter().zip(["X", "Y"].iter()) {
        let mut chart = ChartBuilder::on(&area)
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_ranged(-1.0f64.. 5.0f64, -2.0f64..5.0f64)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .x_desc("t")
            .y_desc(*name)
            .axis_desc_style(("Arial", 20).into_font())
            .draw()?;
        charts.push(chart);
    }
    
   let x_ctx = &mut charts[0];    
    
    x_ctx.configure_mesh().draw()?;
    let x_points: Vec<_> = points.iter().enumerate().map(|(i, p)| (i as f64, p.x)).collect();
    x_ctx.draw_series(x_points.iter().map(|p| Circle::new(*p, 3, RED.filled())))?;
    x_ctx.plotting_area().draw(&PathElement::new(x_points, RED.filled()))?;
           
       let y_ctx = &mut charts[1];    

    y_ctx.configure_mesh().draw()?;
    let x_points: Vec<_> = points.iter().enumerate().map(|(i, p)| (i as f64, p.y)).collect();
    y_ctx.draw_series(x_points.iter().map(|p| Circle::new(*p, 3, RED.filled())))?;
    y_ctx.plotting_area().draw(&PathElement::new(x_points, RED.filled()))?;
    Ok(())
})

X
 
 
t
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
Y
 
 
t
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0